In [1]:
import pandas as pd 
import numpy as np

In [14]:
variants_file = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/ensembl_genomic_positions(in).csv"
df = pd.read_csv(variants_file)
df = df[['chromosome_name', 'start_position', 'end_position']]


# Step 1: Add chromosome prefix
df["chrom"] = "chr" + df["chromosome_name"].astype(str)

# Step 2 & 3: Rename columns
df = df.rename(columns={
    "start_position": "POS",
    "end_position": "POS_INCR"
})

# Optional: keep only the final columns
df = df[['chrom', 'POS', 'POS_INCR']]
df = df.drop_duplicates(subset=["chrom", "POS", "POS_INCR"])
df = df.sort_values(by = ['chrom', 'POS', 'POS_INCR'], 
                ascending = [True, True, True])
print(df.shape)

# save file to a 
file_path = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/ensembl_variants.txt"
df.to_csv(file_path, sep='\t', index=False)

(1134, 3)


In [12]:
variants_file = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/ensembl_genomic_positions(in).csv"
df = pd.read_csv(variants_file)
df = df[['chromosome_name', 'start_position', 'end_position']]


# Step 1: Add chromosome prefix
df["chrom"] = "chr" + df["chromosome_name"].astype(str)

# Step 2 & 3: Rename columns
df = df.rename(columns={
    "start_position": "POS",
})

df['POS_INCR'] = df['POS'] + 1

# Optional: keep only the final columns
df = df[['chrom', 'POS', 'POS_INCR']]
df = df.drop_duplicates(subset=["chrom", "POS", "POS_INCR"])
df = df.sort_values(by = ['chrom', 'POS', 'POS_INCR'], 
                ascending = [True, True, True])
print(df.shape)

# save file to a 
file_path = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/ensembl_variants_increment.txt"
df.to_csv(file_path, sep='\t', index=False)

(1134, 3)


In [9]:
variants_file = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/mart_export.txt"
df = pd.read_csv(variants_file, sep ='\t')
df = df[['Chromosome/scaffold name', 'Start (bp)', 'End (bp)']]
df

# Step 2 & 3: Rename columns
df = df.rename(columns={
    "Chromosome/scaffold name": "chrom",
    "Start (bp)": "POS"
})

df['POS_INCR'] = df['POS'] + 1

# # Step 1: Add chromosome prefix
df["chrom"] = "chr" + df["chrom"].astype(str)

# # Optional: keep only the final columns
df = df[['chrom', 'POS', 'POS_INCR']]
df = df.drop_duplicates(subset=["chrom", "POS", "POS_INCR"])
df = df.sort_values(by = ['chrom', 'POS', 'POS_INCR'], 
                ascending = [True, True, True])
df

# # save file to a 
file_path = "/group/tran3/duytran/ngocduy.tran/Python scripts/bedtools_merging/tested_variants/mart_export.txt"
df.to_csv(file_path, sep='\t', index=False)

/tmp/ipykernel_2854263/4046201363.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(variants_file, sep ='\t')


In [17]:
import os
import pandas as pd
import numpy as np
import subprocess
import os
import pandas as pd
from datetime import date
import argparse


chromosome = 'ensembl_variants'

# function to read the human and encode files
# create promoters txt files with: the first initial rows as the original dataframe
# while the most right column is the column produced by the bed file
def read_cres_files_encode(mastersheet,file_lst, folder, subject, start_ind):
  output_odd = f"Validation merging test/{chromosome}/output_odd.txt"
  output_even = f"Validation merging test/{chromosome}/output_even.txt"
  track_ind = start_ind
  for bed_file in file_lst:
    try:

      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      with open(logging_file, 'a') as fpo:
          fpo.write(f"One missing file: {bed_file}\n")
      pass
    else:
      name = bed_file[: -4]
      print(name)
      # print(bed_file)
      # Run the Bedtools intersect command
      if track_ind == 0:
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      elif(track_ind %2 ==0):
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', output_odd, '-b', os.path.join(folder, bed_file)]

      else:
        output_file=  output_odd
        command = ['bedtools', 'intersect', '-C', '-a', output_even, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)
      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)
      word_count = subprocess.run(["wc", output_file])
      print(word_count.stdout)
      overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
      print(overview.stdout)

      # Display the output file name
      print(track_ind)
      print(f"Intersection saved as {name}")
      with open(logging_file, 'a') as fpo:
          fpo.write(name)
          # fpo.write(word_count.stdout)
          fpo.write(overview.stdout)
          fpo.write(str(track_ind))
      track_ind += 1

  return track_ind


def read_bed_files(mastersheet,subject, start_ind):
  folder = f"BED files/{subject}"
  embryo_folder = f"BED files/{subject}/Clean Embryo files"
  files = os.listdir(folder)

  promoters = sorted([ele for ele in files if 'promoter' in ele])
  enhancers = sorted([ele for ele in files if 'enhancer' in ele])
  # for mice, embryo is a redundant column as it is already either the
  # promtoer or enhancer
  if subject == 'Human':
    embryo = sorted([ele for ele in os.listdir(embryo_folder) if 'embryo' in ele])
  ctcf = sorted([ele for ele in files if 'CTCF' in ele])
  print(promoters)
  ind = read_cres_files_encode(mastersheet,promoters, folder, subject, start_ind = start_ind)
  print(f"Starting ind is: {ind}")
  # investigate_levels(enhancers, folder)
  print(enhancers)
  ind = read_cres_files_encode(mastersheet,enhancers, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  # investigate_levels(embryo, embryo_folder)
  if subject == 'Human':
    print(embryo)
    ind = read_cres_files_encode(mastersheet,embryo, embryo_folder, subject, start_ind = ind)
    print(f"Starting ind is: {ind}")
  print(ctcf)
  # investigate_levels(ctcf, folder)
  ind = read_cres_files_encode(mastersheet,ctcf, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  print(len(promoters), len(enhancers), len(ctcf))
  return ind

# above
validation_file = file_path

bed_starting_ind = read_bed_files(validation_file, subject = "Human", start_ind = 0)
print(f"bed_starting_ind: {bed_starting_ind}")

['heart_left_ventricle_tissue_female_adult_53_years.promoters.bed', 'heart_left_ventricle_tissue_male_adult_34_years.promoters.bed', 'heart_left_ventricle_tissue_male_child_3_years.promoters.bed', 'heart_right_ventricle_tissue_female_adult_46_years.promoters.bed', 'heart_right_ventricle_tissue_male_adult_34_years.promoters.bed', 'heart_right_ventricle_tissue_male_adult_40_years.promoters.bed', 'heart_right_ventricle_tissue_male_child_3_years.promoters.bed']
heart_left_ventricle_tissue_female_adult_53_years.promoters


FileNotFoundError: [Errno 2] No such file or directory: 'bedtools'